# Homework 8
## Due Date:  Tuesday, October 31st at 11:59 PM

# Problem 1:  BST Traversal
This problem builds on Problem 1 of Homework 7 in which you wrote a binary search tree.

### Part 1

As discussed in lecture, three different types to do a depth-first traversal are: preorder, inorder, and postorder. Here is a reference: [Tree Traversal](https://en.wikipedia.org/wiki/Tree_traversal#Depth-first_search).

Write an iterator class called `DFSTraversal` with the following specifications:

* `__init__(self, tree, traversalType)`: Constructor takes a `BinaryTree` object and one of the enums from `DFSTraversalTypes`

```python
from enum import Enum

class DFSTraversalTypes(Enum):
    PREORDER = 1
    INORDER = 2
    POSTORDER = 3
```

* `changeTraversalType(self, traversalType)`: Change the traversal type
* `__iter__(self)`: This is the initialization of an iterator
* `__next__(self)`: This is called in the iterator for getting the next value

Here's how you might use your `DFSTraversal` class:

```python
input_array = [3, 9, 2, 11]
bt = BinaryTree()
for val in input_array:
    bt.insert(val)
traversal = DFSTraversal(bt, DFSTraversalTypes.INORDER)
for val in traversal:
    print(val)
2
3
9
11
```

### Part 2
Put your `BinaryTree` class (from homework 7) and your `DFSTraversal` class (from Part 1 of this homework) in a file titled `TreeTraversal.py`.

In [ ]:
import warnings

# The BinaryNode class for nodes in the BinaryTree
class BinaryNode:
    
    def __init__(self, val):
        self.val = val
        self.p = None
        self.left = None
        self.right = None
    
    def __repr__(self):
        return "BinaryNode({})".format(self.val)
    
    def count_child(self): # count the number of children of this node
        if self.left == None and self.right == None:
            return 0
        elif self.left != None and self.right != None:
            return 2
        else:
            return 1

# The BinaryTree class
class BinaryTree:
    
    def __init__(self):
        self.root = None
    
    def __repr__(self):
        return "BinaryTree()"
    
    # The height of the BinaryTree
    def __len__(self):
        return self.maxDepth(self.root)
    
    # The height of the BinaryTree
    def maxDepth(self, root): 
        if root == None:
            return 0
        else:
            return max(self.maxDepth(root.left), self.maxDepth(root.right))+1
    
    
    # Insert
    def insert(self, val):
        bi_node = BinaryNode(val) # create a new BinaryNode for the value to be inserted
        
        if self.root == None: # if the tree is empty, we just need to insert it at root
            self.root = bi_node
            return
        
        current_node = self.root # walk thru the tree to find the right position to insert
        while current_node != None:
            current_p = current_node
            if val > current_node.val:
                current_node = current_node.right
            else:
                current_node = current_node.left
        
        if val > current_p.val: 
            current_p.right = bi_node # is a right child
        else:
            current_p.left = bi_node # is a left child
        bi_node.p = current_p # set parent
    
    def inOrderWalk(self, node, ordered_nodes):
        if node != None:
            self.inOrderWalk(node.left, ordered_nodes)
            ordered_nodes.append(node.val)
            self.inOrderWalk(node.right, ordered_nodes)
            return ordered_nodes
    
    def preOrderWalk(self, node, ordered_nodes):
        if node != None:
            ordered_nodes.append(node.val)
            self.preOrderWalk(node.left, ordered_nodes)
            self.preOrderWalk(node.right, ordered_nodes)
            return ordered_nodes
    
    def postOrderWalk(self, node, ordered_nodes):
        if node != None:
            self.postOrderWalk(node.left, ordered_nodes)
            self.postOrderWalk(node.right, ordered_nodes)
            ordered_nodes.append(node.val)
            return ordered_nodes
    
    # Delete the nodes with 'None' as value
    def clearNoneNodes(self, node):
        if node != None:
            if node.val == 'None':
                if node == node.p.right:
                    node.p.right = None
                else:
                    node.p.left = None
            self.clearNoneNodes(node.left)
            self.clearNoneNodes(node.right)
    
    # GetValues: calling getValuesNode(self.root, 0, depth, values)
    def getValues(self, depth):
        values = []
        self.getValuesNode(self.root, 0, depth, values)
        self.clearNoneNodes(self.root)
        return values
    
    # GetValues from the subtree rooted at node, store in values
    def getValuesNode(self, node, current_depth, depth, values):
        if node != None:
            if current_depth == depth:
                values.append(node.val)
            else:
                if node.left == None:
                    none_node = BinaryNode('None')
                    none_node.p = node
                    node.left = none_node
                if node.right == None:
                    none_node = BinaryNode('None')
                    none_node.p = node
                    node.right = none_node
                self.getValuesNode(node.left, current_depth+1, depth, values)
                self.getValuesNode(node.right, current_depth+1, depth, values)
    
    # Return the right-most node from the subtree rooted at node
    def tree_max(self, node): 
        while node.right != None:
            node = node.right
        return node

    # Replace the subtree rooted at u with the subtree rooted at v
    def transplant(self, u, v): 
        if u.p == None:
            self.root = v
        elif u == u.p.left:
            u.p.left = v
        else:
            u.p.right = v
        if v != None:
            v.p = u.p
    
    # Search for the value=key thru the subtree rooted at node
    def search(self, node, key):
        while node != None and key != node.val:
            if key > node.val:
                node = node.right
            else:
                node = node.left
        return node
    
    # Remove
    def remove(self, val):
        rm_node = self.search(self.root, val)
        if rm_node == None: # invalid remove node
            warnings.warn('The value to be removed does not has a node associated.')
            return
        if rm_node.left == None:
            self.transplant(rm_node, rm_node.right)
        elif rm_node.right == None:
            self.transplant(rm_node, rm_node.left)
        else:
            left_max = self.tree_max(rm_node.left)
            if left_max.p != rm_node:
                self.transplant(left_max, left_max.left)
                left_max.left = rm_node.left
                left_max.left.p = left_max
            self.transplant(rm_node, left_max)
            left_max.right = rm_node.right
            left_max.right.p = left_max


In [ ]:
from enum import Enum

class DFSTraversalTypes(Enum):
    PREORDER = 1
    INORDER = 2
    POSTORDER = 3

class DFSTraversal:
    
    # DFSTraversal Constructor
    def __init__(self, tree, traversalType):
        if traversalType == DFSTraversalTypes.INORDER:
            self.ordered_nodes = tree.inOrderWalk(tree.root, list())
        elif traversalType == DFSTraversalTypes.PREORDER:
            self.ordered_nodes = tree.preOrderWalk(tree.root, list())
        elif traversalType == DFSTraversalTypes.POSTORDER:
            self.ordered_nodes = tree.postOrderWalk(tree.root, list())
        else:
            raise TypeError('TraversalType Wrong: must be DFSTraversalTypes.INORDER/PREORDER/POSTORDER')
        # set attributes
        self.tree = tree 
        self.type = traversalType
        self.index = 0
    
    # Change Traversal Type
    def changeTraversalType(self, traversalType):
        if self.type == traversalType: # nothing changed
            return
        else:
            if traversalType == DFSTraversalTypes.INORDER: # change to INORDER
                self.ordered_nodes = self.tree.inOrderWalk(self.tree.root, list())
            elif traversalType == DFSTraversalTypes.PREORDER: # change to PREORDER
                self.ordered_nodes = self.tree.preOrderWalk(self.tree.root, list())
            elif traversalType == DFSTraversalTypes.POSTORDER: # change to POSTORDER
                self.ordered_nodes = self.tree.postOrderWalk(self.tree.root, list())
            else:
                raise TypeError('TraversalType Wrong: must be DFSTraversalTypes.INORDER/PREORDER/POSTORDER')
            print('Changed traversalType to be {}'.format(traversalType))
            self.type = traversalType
            self.index = 0
    
    # Initialize the iterator
    def __iter__(self):
        return self
    
    # Called by __iter__ to get the next value
    def __next__(self):
        try:
            node = self.ordered_nodes[self.index] 
        except IndexError:
            raise StopIteration() 
        self.index += 1
        return node 
            
            
    

In [1]:
# Using codes from imported modules
from TreeTraversal import *

tree1 = BinaryTree()
arr1 = [20, 10, 17, 14, 3, 0]
for a1 in arr1:
    tree1.insert(a1)

tree1.postOrderWalk(tree1.root, list())

[0, 3, 14, 17, 10, 20]

In [2]:
print('Height of tree1: ', len(tree1))
for i in range(len(tree1)):
    print('Level %d values: ' % i, tree1.getValues(i))

Height of tree1:  4
Level 0 values:  [20]
Level 1 values:  [10, 'None']
Level 2 values:  [3, 17, 'None', 'None']
Level 3 values:  [0, 'None', 14, 'None', 'None', 'None', 'None', 'None']


In [3]:
input_array = [20, 10, 17, 14, 3, 0]
bt = BinaryTree()
for val in input_array:
    bt.insert(val)
traversal = DFSTraversal(bt, DFSTraversalTypes.INORDER)

for val in traversal:
    print(val)

0
3
10
14
17
20


In [4]:
traversal.changeTraversalType(DFSTraversalTypes.PREORDER)
for val in traversal:
    print(val)

Changed traversalType to be DFSTraversalTypes.PREORDER
20
10
3
0
17
14


In [5]:
traversal.changeTraversalType(DFSTraversalTypes.POSTORDER)
for val in traversal:
    print(val)

Changed traversalType to be DFSTraversalTypes.POSTORDER
0
3
14
17
10
20


---

## Problem 2: Markov Chains

[Markov Chains](https://en.wikipedia.org/wiki/Markov_chain) are widely used to model and predict discrete events. Underlying Markov chains are Markov processes which make the assumption that the outcome of a future event only depends on the event immediately preceeding it. In this exercise, we will be assuming that weather has Markov properties (e.g. today's weather is dependent only on yesterday's weather). We will use the Markov assumption to create a basic model for predicting weather.

To begin, let's categorize weather into 7 types: ['sunny', 'cloudy', 'rainy', 'snowy', 'windy', 'hailing'].

In the `weather.csv` file accompanying this homework, each row corresponds to one type of weather (in the order given above) and each column is the probability of one type of weather occurring the following day (also in the order given above).

The $ij$th element is the probability that the $j$th weather type occurs after the $i$th weather type. So for example, (1,2) is the probability a cloudy day occurs after a sunny day.

Take a look at the data. Make sure you see how if the previous day was sunny, the following day will have a 0.4 probability of being sunny as well. If the previous day was raining (index $i = 3$), then the following day (index $j$) has a 0.05 probability of being windy ($j = 5$).

### Part 1:  Parse the `.csv` file into a `Numpy` array

In [6]:
import numpy as np

#Load CSV file -- hint: you can use np.genfromtxt()
weather_arr = np.genfromtxt('weather.csv', delimiter=',')
weather_arr

array([[ 0.4 ,  0.3 ,  0.1 ,  0.05,  0.1 ,  0.05],
       [ 0.3 ,  0.4 ,  0.1 ,  0.1 ,  0.08,  0.02],
       [ 0.2 ,  0.3 ,  0.35,  0.05,  0.05,  0.05],
       [ 0.1 ,  0.2 ,  0.25,  0.3 ,  0.1 ,  0.05],
       [ 0.15,  0.2 ,  0.1 ,  0.15,  0.3 ,  0.1 ],
       [ 0.1 ,  0.2 ,  0.35,  0.1 ,  0.05,  0.2 ]])

### Part 2:  Create a class called `Markov` that has the following methods:

* `load_data(array)`: loads the Numpy 2D array and stores it as a class variable.
* `get_prob(previous_day, following_day)`: returns the probability of `following_day` weather given `previous_day` weather. 

**Note:** `previous_day` and `following_day` should be passed in string form (e.g. "sunny"), as opposed to an index (e.g. 0). 




In [7]:
class Markov:
    
    def __init__(self, state0='sunny'): # Initial state default to sunny
        self.data = None
        self.weather_types = ['sunny', 'cloudy', 'rainy', 'snowy', 'windy', 'hailing']
        self.weather_dict = {t : i for i, t in enumerate(self.weather_types)}
        self.index = self.weather_dict[state0]
        
    def load_data(self, array):
        self.data = array
        
    def get_prob(self, previous_day, following_day):
        try:
            p_i, f_i = self.weather_dict[previous_day], self.weather_dict[following_day]
            return float("{0:.4f}".format(self.data[p_i, f_i]))
        except KeyError as e:
            print('KeyError {}: Key must in set([sunny, cloudy, rainy, snowy, windy, hailing])'.format(e))

In [8]:
mk2 = Markov()
mk2.load_data(weather_arr)
mk2.get_prob('sunny', 's')

KeyError 's': Key must in set([sunny, cloudy, rainy, snowy, windy, hailing])


In [9]:
mk2.get_prob('sunny', 'sunny')

0.4

In [10]:
mk2.get_prob('rainy', 'windy')

0.05

In [11]:
mk2.get_prob('hailing', 'sunny')

0.1

---

## Problem 3: Iterators

Iterators are a convenient way to walk along your Markov chain.

#### Part 1: Using your `Markov` class from Problem 3, write `Markov` as an iterator by implementing the `__iter__()` and `__next__()` methods.

Remember:  
* `__iter__()` should return the iterator object and should be implicitly called when the loop begins
* The `__next()__` method should return the next value and is implicitly called at each step in the loop.

Each 'next' step should be stochastic (i.e. randomly selected based on the relative probabilities of the following day weather types) and should return the next day's weather as a string (e.g. "sunny") rather than an index (e.g. 0).

In [12]:
# Class of Markov as an iterator
class Markov:
    
    # Constructor of the Markov Iterator
    def __init__(self, state0='sunny'): # Initial state default to sunny
        self.data = None
        self.weather_types = ['sunny', 'cloudy', 'rainy', 'snowy', 'windy', 'hailing']
        self.weather_dict = {t : i for i, t in enumerate(self.weather_types)}
        self.index = self.weather_dict[state0]
        #print(self.weather_types, '\n')
    
    # Load weather.csv 
    def load_data(self, array):
        self.data = array
    
    # Get probability of the following_day weather given the previous_day weather
    def get_prob(self, previous_day, following_day):
        try:
            p_i, f_i = self.weather_dict[previous_day], self.weather_dict[following_day]
            return float("{0:.4f}".format(self.data[p_i, f_i]))
        except KeyError as e:
            print('KeyError {}: Key must in set([sunny, cloudy, rainy, snowy, windy, hailing])'.format(e))
    
    # Return the Markov iterator itself
    def __iter__():
        return self
    
    # Called by __iter__ to get the next value
    def __next__(self):
        next_probs = self.data[self.index, :]
        next_probs_int = (next_probs * 100).astype(np.int8)
        next_cum_int = np.zeros(next_probs_int.shape).astype(np.int8)
        
        # Randomly choosing the nextday's weather using cumulant probabilities as boundaries
        for i, next_prob in enumerate(next_probs_int):
            if i == 0:
                next_cum_int[i] = next_prob
            else:
                next_cum_int[i] = next_cum_int[i-1] + next_prob
        r = np.random.choice(100)
        print('------------------ r={}, next_cum_int={}'.format(r, next_cum_int))
        if r < next_cum_int[0]:
            self.index = 0
        else:
            idx = 1
            while idx < len(next_cum_int):
                if r >= next_cum_int[idx-1] and r < next_cum_int[idx]:
                    break
                idx += 1
            self.index = idx
        print('------------------ the_next_index = {}, {}'.format(self.index, self.weather_types[self.index]))
        return self.weather_types[self.index]
    

In [13]:
np.random.seed(12345)
mk = Markov('sunny')
mk.load_data(weather_arr)
for i in range(10):
    print(next(mk))

------------------ r=98, next_cum_int=[ 40  70  80  85  95 100]
------------------ the_next_index = 5, hailing
hailing
------------------ r=29, next_cum_int=[ 10  30  65  75  80 100]
------------------ the_next_index = 1, cloudy
cloudy
------------------ r=1, next_cum_int=[ 30  70  80  90  98 100]
------------------ the_next_index = 0, sunny
sunny
------------------ r=36, next_cum_int=[ 40  70  80  85  95 100]
------------------ the_next_index = 0, sunny
sunny
------------------ r=41, next_cum_int=[ 40  70  80  85  95 100]
------------------ the_next_index = 1, cloudy
cloudy
------------------ r=34, next_cum_int=[ 30  70  80  90  98 100]
------------------ the_next_index = 1, cloudy
cloudy
------------------ r=29, next_cum_int=[ 30  70  80  90  98 100]
------------------ the_next_index = 0, sunny
sunny
------------------ r=1, next_cum_int=[ 40  70  80  85  95 100]
------------------ the_next_index = 0, sunny
sunny
------------------ r=59, next_cum_int=[ 40  70  80  85  95 100]
--------

## Note of Discussion
> After discussion with Michelle (Chia Chi Ho), I tried using 

> **`np.random.choice(list, size=1, p=specified_probs)[0]` **

> to directly implement the random choice by specified probabilities. The codes get shorter and cleaner.

> Codes below this part use `__next__(self)` implemented with **`np.random.choice(list, size=1, p=specified_probs)[0]`**

In [14]:
# Class of Markov as an iterator
class Markov:
    
    # Constructor of the Markov Iterator
    def __init__(self, state0='sunny'): # Initial state default to sunny
        self.data = None
        self.weather_types = ['sunny', 'cloudy', 'rainy', 'snowy', 'windy', 'hailing']
        self.weather_dict = {t : i for i, t in enumerate(self.weather_types)}
        self.index = self.weather_dict[state0]
        #print(self.weather_types, '\n')
    
    # Load weather.csv 
    def load_data(self, array):
        self.data = array
    
    # Get probability of the following_day weather given the previous_day weather
    def get_prob(self, previous_day, following_day):
        try:
            p_i, f_i = self.weather_dict[previous_day], self.weather_dict[following_day]
            return float("{0:.4f}".format(self.data[p_i, f_i]))
        except KeyError as e:
            print('KeyError {}: Key must in set([sunny, cloudy, rainy, snowy, windy, hailing])'.format(e))
    
    # Return the Markov iterator itself
    def __iter__():
        return self
    
    # Called by __iter__ to get the next value, using np.random.choice
    def __next__(self):
        next_probs = self.data[self.index, :]
        next_weather = np.random.choice(self.weather_types, size=1, p=next_probs)[0]
        self.index = self.weather_dict[next_weather]
        return next_weather
    

In [15]:
# Using __next__ implemented with np.random.choice(self.weather_types, size=1, p=next_probs)[0]
np.random.seed(12345)
mk = Markov('sunny')
mk.load_data(weather_arr)
for i in range(10):
    print(next(mk))

windy
cloudy
sunny
sunny
cloudy
cloudy
windy
windy
windy
windy


#### Part 2: We want to predict what weather will be like in a week for 5 different cities.

Now that we have our `Markov` iterator, we can try to predict what the weather will be like in seven days from now.

Given each city's current weather in the dictionary `city_weather` (see below), simulate what the weather will be like in 7 days from now.  Rather than just producing one prediction per city, simulate 100 such predictions per city and store the most commonly occuring prediction.

In your submission, print a dictionary `city_weather_predictions` that has each city as a key and the most commonly predicted weather as the corresponding value.

**Note**: Don't worry if your values don't seem to make intuitive sense.  We made up the weather probabilities.

In [16]:
city_weather = {
    'New York': 'rainy',
    'Chicago': 'snowy',
    'Seattle': 'rainy',
    'Boston': 'hailing',
    'Miami': 'windy',
    'Los Angeles': 'cloudy',
    'San Fransisco': 'windy'
}

np.random.seed(12345)
n_days = 7
n_sim = 100
city_weather_predictions = {}
city_weather_predictions_sims = {}

for city, w0 in city_weather.items():
    sim_preds_count = np.zeros(6).astype(np.int8)
    for i in range(n_sim): # In each simulation,
        mk = Markov(w0) # Initialize the Markov Chain
        mk.load_data(weather_arr) # Load the transfer probs
        for ii in range(n_days): # Predict for 7 consecutive days
            w_pred = next(mk)
        sim_preds_count[mk.index] += 1
        
#     print('~~~~~~~ sim_preds: {}, np.argmax(sim_preds_count): {}'.format(sim_preds_count, np.argmax(sim_preds_count)))
    
    predicted = mk.weather_types[np.argmax(sim_preds_count)]
    city_weather_predictions[city] = predicted
    city_weather_predictions_sims[city] = sim_preds_count
#     print('np.sum(sim_preds_count) = {}'.format(np.sum(sim_preds_count)))
    print('The weather in 7 days from now in {}: {}\n'.format(city, predicted))
    


The weather in 7 days from now in New York: cloudy

The weather in 7 days from now in Chicago: cloudy

The weather in 7 days from now in Seattle: sunny

The weather in 7 days from now in Boston: sunny

The weather in 7 days from now in Miami: sunny

The weather in 7 days from now in Los Angeles: cloudy

The weather in 7 days from now in San Fransisco: sunny



In [17]:
for (city, w_pred), (c, counts) in zip(city_weather_predictions.items(), city_weather_predictions_sims.items()):
    print('{}: {}  {}'.format(city, w_pred, counts))

New York: cloudy  [25 32 22 13  7  1]
Chicago: cloudy  [29 32 18  4 12  5]
Seattle: sunny  [30 21 22 12 10  5]
Boston: sunny  [30 28 11 10 15  6]
Miami: sunny  [33 30 14 10  9  4]
Los Angeles: cloudy  [26 31 14 10  8 11]
San Fransisco: sunny  [29 28 16 10  6 11]


In [18]:
# Print the dictionary  city_weather_predictions
print(city_weather_predictions)

{'New York': 'cloudy', 'Chicago': 'cloudy', 'Seattle': 'sunny', 'Boston': 'sunny', 'Miami': 'sunny', 'Los Angeles': 'cloudy', 'San Fransisco': 'sunny'}
